In [1]:
import os

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl

class MNISTModel(pl.LightningModule):

    def __init__(self):
        super(MNISTModel, self).__init__()
        # not the best model...
        self.l1 = torch.nn.Linear(28 * 28, 10)

    def forward(self, x):
        # called with self(x)
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch, batch_nb):
        # REQUIRED
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_nb):
        # OPTIONAL
        x, y = batch
        y_hat = self(x)
        return {'val_loss': F.cross_entropy(y_hat, y)}

    def validation_epoch_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'val_loss': avg_loss, 'log': tensorboard_logs}

    def test_step(self, batch, batch_nb):
        # OPTIONAL
        x, y = batch
        y_hat = self(x)
        return {'test_loss': F.cross_entropy(y_hat, y)}

    def test_epoch_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        logs = {'test_loss': avg_loss}
        return {'test_loss': avg_loss, 'log': logs, 'progress_bar': logs}

    def configure_optimizers(self):
        # REQUIRED
        # can return multiple optimizers and learning_rate schedulers
        # (LBFGS it is automatically supported, no need for closure function)
        return torch.optim.Adam(self.parameters(), lr=0.02)

    def train_dataloader(self):
        # REQUIRED
        return DataLoader(MNIST(os.getcwd(), train=True, download=True, transform=transforms.ToTensor()), batch_size=32)

    def val_dataloader(self):
        # OPTIONAL
        return DataLoader(MNIST(os.getcwd(), train=True, download=True, transform=transforms.ToTensor()), batch_size=32)

    def test_dataloader(self):
        # OPTIONAL
        return DataLoader(MNIST(os.getcwd(), train=False, download=True, transform=transforms.ToTensor()), batch_size=32)

In [4]:
mnist_model = MNISTModel()
   
# most basic trainer, uses good defaults (1 gpu)
trainer = pl.Trainer(gpus=1,max_epochs=5)    
trainer.fit(mnist_model)

.44it/s, loss=1.462, v_num=8]
Epoch 1:  50%|█████     | 1875/3750 [00:12<00:12, 150.45it/s, loss=1.469, v_num=8]
Validating: 0it [00:00, ?it/s]
Epoch 2:  50%|█████     | 1875/3750 [00:12<00:12, 147.60it/s, loss=1.448, v_num=8]
Validating: 0it [00:00, ?it/s]
Epoch 3:  50%|█████     | 1875/3750 [00:13<00:13, 141.53it/s, loss=1.469, v_num=8]
Validating: 0it [00:00, ?it/s]
Epoch 4:  50%|█████     | 1875/3750 [00:12<00:12, 144.53it/s, loss=1.444, v_num=8]
Validating: 0it [00:00, ?it/s]
Epoch 4: 100%|██████████| 3750/3750 [00:20<00:00, 180.57it/s, loss=1.444, v_num=8]


1

In [5]:
trainer.test()

Testing:  90%|█████████ | 282/313 [00:00<00:00, 320.76it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': tensor(1.5283, device='cuda:0')}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 313/313 [00:01<00:00, 310.21it/s]


[{'test_loss': 1.528316617012024}]